<a href="https://colab.research.google.com/github/ErickQuinteroOsorio/edp_con_pinn/blob/main/notebooks/analisis_edp_con_pinn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
# We set seeds initially. This feature controls the randomization of
# variables (e.g. initial weights of the network).
# By doing it so, we can reproduce same results.
tf.random.set_seed(123)
# 100 equidistant points in the domain are created
x = tf.linspace(0.0, 1.0, 100)
# boundary conditions T(0)=T(1)=0 and \kappa are introduced.
bcs_x = [0.0, 1.0]
bcs_T = [0.0, 0.0]

In [ ]:
bcs_x_tensor = tf.convert_to_tensor(bcs_x)[:, None]
bcs_T_tensor = tf.convert_to_tensor(bcs_T)[:, None]
kappa = 0.5

# Number of iterations
N = 1000
# ADAM optimizer with learning rate of 0.005
optim = tf.keras.optimizers.Adam(learning_rate=0.005)

# Function for creating the model
def buildModel(num_hidden_layers, num_neurons_per_layer):
  tf.keras.backend.set_floatx("float32")

  # Initialize a feedforward neural network
  model = tf.keras.Sequential()

  # Input is one dimensional ( one spatial dimension)
  model.add(tf.keras.Input(1))

  # Append hidden layers
  for _ in range(num_hidden_layers):
    model.add(tf.keras.layers.Dense(num_neurons_per_layer,
                                    activation=tf.keras.activations.get("tanh"),
                                    kernel_initializer="glorot_normal",
                                    )
    )

  # Output is one-dimensional
  model.add(tf.keras.layers.Dense(1))
  return model


# determine the model size (3 hidden layers with 32 neurons each)
model = buildModel(3, 32)